In [1]:
import pandas as pd
import datetime
import pytz
import os

In [7]:
dfm = pd.read_csv('tmall/raw/dataset15.csv', sep='\t')
record_num = len(dfm)
print(record_num)
dfm.head(3)

45881450


,UserId,ItemId,SessionId,Time
0,1,181459,0,1.444342e+09
1,1,779078,0,1.444342e+09
2,1,779078,0,1.444342e+09


In [8]:
dfm['date'] = dfm.Time.map(lambda x:datetime.date.fromtimestamp(x))
dfm.head(3)

,UserId,ItemId,SessionId,Time,date
0,1,181459,0,1.444342e+09,2015-10-09
1,1,779078,0,1.444342e+09,2015-10-09
2,1,779078,0,1.444342e+09,2015-10-09


In [141]:
start_date = datetime.date(2015,5,20)
end_date = datetime.date(2015,11,12)

target_start_date = start_date + datetime.timedelta(28)

RF2N = {}
RF2H = {}
for i in range((end_date - target_start_date).days + 1):
    target_date = target_start_date + datetime.timedelta(i)
    train_start_date = target_date - datetime.timedelta(28) 
    train_end_date = target_date - datetime.timedelta(1) 
    if i>146:break
    print(i, train_start_date.strftime("%F"), '~', train_end_date.strftime("%F"), ':', target_date.strftime("%F"), flush=True)  
    x_dfm = dfm[(train_start_date <= dfm.date) & (dfm.date <= train_end_date)].copy()
    y_dfm = dfm[dfm.date == target_date].copy()
    x_record_num = len(x_dfm)
    y_record_num = len(y_dfm)
    print(x_record_num, y_record_num)
    x_dfm['target_date'] = target_date
    x_dfm['day_rcen'] = (x_dfm.target_date - x_dfm.date).map(lambda x:x.days)

    U2Is = {}
    for row in y_dfm.itertuples():
        U2Is.setdefault(row.UserId, set())
        U2Is[row.UserId].add(row.ItemId)

    U2I2Info = {}
    for row in x_dfm.itertuples():
        U2I2Info.setdefault(row.UserId, {})
        U2I2Info[row.UserId].setdefault(row.ItemId, [])
        U2I2Info[row.UserId][row.ItemId].append(row.day_rcen)

    for UserId, I2Info in U2I2Info.items():
        HitSet = set(U2Is[UserId]) if UserId in U2Is else set()
        for ItemId, Info in I2Info.items():
            freq = len(Info)
            rcen = min(Info)
            RF2N.setdefault((rcen,freq), 0)
            RF2H.setdefault((rcen,freq), 0)
            RF2N[rcen,freq] += 1
            if ItemId in HitSet:
                RF2H[rcen,freq] += 1
    N = 0
    for rf, n in RF2N.items():
        N += n
    print('data:', N)
RF2P = {}
for rf, n in RF2N.items():
    RF2P[rf] = RF2H[rf] / n                    

0 2015-05-20 ~ 2015-06-16 : 2015-06-17
4420639 282989
data: 3001219
1 2015-05-21 ~ 2015-06-17 : 2015-06-18
4519295 223902
data: 6056074
2 2015-05-22 ~ 2015-06-18 : 2015-06-19
4550569 167593
data: 9118595
3 2015-05-23 ~ 2015-06-19 : 2015-06-20
4504898 164926
data: 12153786
4 2015-05-24 ~ 2015-06-20 : 2015-06-21
4473374 157829
data: 15171128
5 2015-05-25 ~ 2015-06-21 : 2015-06-22
4450687 152291
data: 18175144
6 2015-05-26 ~ 2015-06-22 : 2015-06-23
4445567 150328
data: 21179674
7 2015-05-27 ~ 2015-06-23 : 2015-06-24
4442178 185002
data: 24186804
8 2015-05-28 ~ 2015-06-24 : 2015-06-25
4456661 231917
data: 27207943
9 2015-05-29 ~ 2015-06-25 : 2015-06-26
4513306 489460
data: 30269922
10 2015-05-30 ~ 2015-06-26 : 2015-06-27
4833680 283377
data: 33519418
11 2015-05-31 ~ 2015-06-27 : 2015-06-28
4958526 255142
data: 36846893
12 2015-06-01 ~ 2015-06-28 : 2015-06-29
5062245 125720
data: 40238767
13 2015-06-02 ~ 2015-06-29 : 2015-06-30
5046425 113273
data: 43618791
14 2015-06-03 ~ 2015-06-30 : 2015

In [142]:
Freq = []
Rcen = []
for rf, p in RF2P.items():
    r, f = rf 
    Rcen.append(r)
    Freq.append(f)
R = list(range(1, max(Rcen)))
F = list(range(1, max(Freq)))
for r in R:
    for f in F:
        if not (r,f) in RF2P:
            RF2P[r,f] = 0.0

In [150]:
g = open('fr2p.csv', 'w')
g.write('freq,rcen,prob\n')
for rf, p in sorted(RF2P.items()):
    r, f = rf
    if f > 40:continue
    print(f, r, p)
    g.write('%d,%d,%f\n' % (f, r, p))
g.close()

1 1 0.017500690171067944
2 1 0.04947213185043968
3 1 0.07770112254443405
4 1 0.09698548439615431
5 1 0.11165173690027824
6 1 0.12480112039480579
7 1 0.13577717722953536
8 1 0.1457378944213726
9 1 0.15583581747305664
10 1 0.16462814865500577
11 1 0.17291806532589976
12 1 0.1799306625577812
13 1 0.187766421858148
14 1 0.19613424240418348
15 1 0.20426468099509223
16 1 0.21441557052721635
17 1 0.21944480989343507
18 1 0.22109400589584016
19 1 0.22947199357558723
20 1 0.22695035460992907
21 1 0.2433679354094579
22 1 0.238061797752809
23 1 0.2648648648648649
24 1 0.2693089430894309
25 1 0.2805299539170507
26 1 0.29318651066758433
27 1 0.2946495855312736
28 1 0.2775
29 1 0.26325940212150434
30 1 0.27491408934707906
31 1 0.29907773386034253
32 1 0.32
33 1 0.27960526315789475
34 1 0.29924242424242425
35 1 0.3006535947712418
36 1 0.2842377260981912
37 1 0.32642487046632124
38 1 0.2857142857142857
39 1 0.28753993610223644
40 1 0.27586206896551724
1 2 0.009920405963540484
2 2 0.027138796918814826


In [95]:
target_date = target_start_date + datetime.timedelta(i)
train_start_date = target_date - datetime.timedelta(28) 
train_end_date = target_date - datetime.timedelta(1) 
print(train_start_date.strftime("%F"), '~', train_end_date.strftime("%F"), ':', target_date.strftime("%F"))  
x_dfm = dfm[(train_start_date <= dfm.date) & (dfm.date <= train_end_date)].copy()
y_dfm = dfm[dfm.date == target_date].copy()
x_record_num = len(x_dfm)
y_record_num = len(y_dfm)
print(x_record_num, y_record_num)
x_dfm['target_date'] = target_date
x_dfm['day_rcen'] = (x_dfm.target_date - x_dfm.date).map(lambda x:x.days)
x_dfm.head()

2015-05-20 ~ 2015-06-16 : 2015-06-17
4420639 282989


,UserId,ItemId,SessionId,Time,date,target_date,day_rcen
26,2,348983,5,1.432678e+09,2015-05-27,2015-06-17,21
27,2,749563,5,1.432678e+09,2015-05-27,2015-06-17,21
130,4,836727,26,1.432678e+09,2015-05-27,2015-06-17,21
131,4,243874,26,1.432678e+09,2015-05-27,2015-06-17,21
132,4,185489,26,1.432678e+09,2015-05-27,2015-06-17,21


In [104]:
U2Is = {}
for row in y_dfm.itertuples():
    U2Is.setdefault(row.UserId, [])
    U2Is[row.UserId].append(row.ItemId)
for UserId, Is in U2Is.items():
    U2Is[UserId] = set(Is)
print(len(U2Is))
U2Is[8]

30844


{37152, 599824}

In [106]:
U2I2Info = {}
for row in x_dfm.itertuples():
    U2I2Info.setdefault(row.UserId, {})
    U2I2Info[row.UserId].setdefault(row.ItemId, [])
    U2I2Info[row.UserId][row.ItemId].append(row.day_rcen)
len(U2I2Info)

217318

In [109]:
RF2N = {}
RF2H = {}
for UserId, I2Info in U2I2Info.items():
    if UserId in U2Is:
        HitSet = U2Is[UserId]
    else:
        HitSet = set()
    for ItemId, Info in I2Info.items():
        freq = len(Info)
        rcen = min(Info)
        RF2N.setdefault((rcen,freq), 0)
        RF2H.setdefault((rcen,freq), 0)
        RF2N[rcen,freq] += 1
        if ItemId in HitSet:
            RF2H[rcen,freq] += 1
print(len(RF2N), len(RF2H))

1150 1150


In [125]:
RF2P = {}
for rf, n in RF2N.items():
    RF2P[rf] = RF2H[rf] / n

In [126]:
RF2P

{(21, 1): 0.0017676199054267056,
 (21, 3): 0.0033919864320542717,
 (21, 2): 0.003389112476170303,
 (28, 1): 0.001556021804382962,
 (28, 3): 0.002331002331002331,
 (9, 1): 0.0035650394470971313,
 (25, 2): 0.003065721402567542,
 (25, 1): 0.0015261387717013044,
 (9, 4): 0.018808777429467086,
 (9, 5): 0.014669926650366748,
 (8, 1): 0.004289663085428184,
 (4, 2): 0.025625422582826234,
 (13, 1): 0.0024156760522235456,
 (4, 1): 0.010230521257831032,
 (24, 2): 0.0029993183367416497,
 (24, 1): 0.0012772066119191878,
 (24, 3): 0.0031081641110650643,
 (24, 4): 0.003147482014388489,
 (27, 3): 0.0033549541489599644,
 (27, 1): 0.0015022727933551082,
 (26, 1): 0.0015339804052264903,
 (25, 5): 0.007252215954875101,
 (26, 3): 0.004787437763309077,
 (23, 1): 0.0013508828557424477,
 (23, 2): 0.0032429928190873293,
 (21, 8): 0.006134969325153374,
 (21, 6): 0.012295081967213115,
 (18, 1): 0.001903198563085085,
 (15, 1): 0.002034267377227666,
 (15, 3): 0.004975124378109453,
 (3, 1): 0.014423316863801966,
 (

In [120]:
RF2H[21,2] / RF2N[21,2]

0.003389112476170303

In [80]:
#RF2H
#RF2N

In [50]:
R = list(range(1,29))
print(R)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


In [59]:
Freq = []
for rf, n in RF2N.items():
    r, f = rf
    Freq.append(f)
F = list(range(1, max(F)))
print(max(F))

792


In [81]:
RF2P = {}
for r in R:
    for f in F:
        #if not (r,f) in RF2N:RF2N[r,f] = 0
        n = RF2N[r,f]
        #print(r,f,n)
        if (r,f) not in RF2H:
            RF2H[r,f] = 0
        h = RF2H[r,f]
        if n != 0:
            RF2P[r,f] = 1.0 * h/n
        else:
            RF2P[r,f] = 0

In [82]:
for r in R:
    for f in F:
        #print(r, f, RF2P[r,f])
        pass

In [89]:
RF2N = {}
RF2H = {}
for UserId, I2Info in U2I2Info.items():
    HitSet = set(U2Is[UserId]) if UserId in U2Is else set()
    for ItemId, Info in I2Info.items():
        freq = len(Info)
        rcen = min(Info)
        RF2N.setdefault((rcen,freq), 0)
        RF2H.setdefault((rcen,freq), 0)
        RF2N[rcen,freq] += 1
        if ItemId in HitSet:
            RF2H[rcen,freq] += 1
        break

In [91]:
x_dfm.head(100)

,UserId,ItemId,SessionId,Time,date,target_date,day_rcen
26,2,348983,5,1.432678e+09,2015-05-27,2015-06-17,21
27,2,749563,5,1.432678e+09,2015-05-27,2015-06-17,21
130,4,836727,26,1.432678e+09,2015-05-27,2015-06-17,21
131,4,243874,26,1.432678e+09,2015-05-27,2015-06-17,21
132,4,185489,26,1.432678e+09,2015-05-27,2015-06-17,21
...,...,...,...,...,...,...,...
798,14,22374,200,1.432246e+09,2015-05-22,2015-06-17,26
799,14,872614,200,1.432246e+09,2015-05-22,2015-06-17,26
800,14,451932,201,1.432332e+09,2015-05-23,2015-06-17,25
801,14,413074,201,1.432332e+09,2015-05-23,2015-06-17,25
